# Cut-list calculation for fire organ

In [12]:
from pint import UnitRegistry
unit = UnitRegistry()

In [17]:
speed_of_sound = 258 * unit.meter

scale = {
    'A1': 55,
    'C2': 65.41,
    'D2': 73.42,
    'Ef2': 77.78,
    'E2': 82.41,
    'G2': 98,
    'A2': 110,
    'C3': 130.81,
    'D3': 146.83,
    'Ef3': 155.56,
    'E3': 164.81,
    'G3': 196
}



def wave_len(frequency: float) -> float:
    return speed_of_sound / frequency

def pipe_len(frequency: float) -> float:
    return speed_of_sound / frequency / 2

theoretical_lens = {k: pipe_len(v) for k, v in scale.items()}
theoretical_lens

{'A1': 2.3454545454545452 <Unit('meter')>,
 'C2': 1.9721755083320593 <Unit('meter')>,
 'D2': 1.7570144374829746 <Unit('meter')>,
 'Ef2': 1.6585240421702236 <Unit('meter')>,
 'E2': 1.5653440116490718 <Unit('meter')>,
 'G2': 1.316326530612245 <Unit('meter')>,
 'A2': 1.1727272727272726 <Unit('meter')>,
 'C3': 0.9861631373748184 <Unit('meter')>,
 'D3': 0.8785670503303139 <Unit('meter')>,
 'Ef3': 0.8292620210851118 <Unit('meter')>,
 'E3': 0.7827194951762636 <Unit('meter')>,
 'G3': 0.6581632653061225 <Unit('meter')>}

In [25]:
# Compensate for diameter
internal_diameter_in = 2.0 * unit.inch
id_m = internal_diameter_in.to(unit.centimeter)
d_comp = id_m * 2
print(d_comp)
print(d_comp.to('inch'))

# Compensation for internal bushing
bushing_len = 0 * unit.meter

def compensation(plen: float) -> float:
    return plen + bushing_len - d_comp

comp_len = {k: compensation(v) for k, v in theoretical_lens.items()}
comp_len

10.16 centimeter
4.0 inch


{'A1': 2.2438545454545453 <Unit('meter')>,
 'C2': 1.8705755083320594 <Unit('meter')>,
 'D2': 1.6554144374829747 <Unit('meter')>,
 'Ef2': 1.5569240421702237 <Unit('meter')>,
 'E2': 1.4637440116490719 <Unit('meter')>,
 'G2': 1.214726530612245 <Unit('meter')>,
 'A2': 1.0711272727272727 <Unit('meter')>,
 'C3': 0.8845631373748184 <Unit('meter')>,
 'D3': 0.7769670503303139 <Unit('meter')>,
 'Ef3': 0.7276620210851118 <Unit('meter')>,
 'E3': 0.6811194951762636 <Unit('meter')>,
 'G3': 0.5565632653061224 <Unit('meter')>}

In [27]:
{k: v.to('inch') for k, v in comp_len.items()}

{'A1': 88.34073013600572 <Unit('inch')>,
 'C2': 73.64470505244329 <Unit('inch')>,
 'D2': 65.17379675129821 <Unit('inch')>,
 'Ef2': 61.296222132685976 <Unit('inch')>,
 'E2': 57.62771699405795 <Unit('inch')>,
 'G2': 47.8238791579624 <Unit('inch')>,
 'A2': 42.170365068002866 <Unit('inch')>,
 'C3': 34.825320369087336 <Unit('inch')>,
 'D3': 30.58925395001236 <Unit('inch')>,
 'Ef3': 28.648111066342985 <Unit('inch')>,
 'E3': 26.81572815654581 <Unit('inch')>,
 'G3': 21.911939578981197 <Unit('inch')>}

In [6]:
# Calculate cult-list
# Minimize waste using binpacking algorithm

import binpacking

needed = pipe_lengths
print(binpacking.to_constant_volume(needed, 100))

ModuleNotFoundError: No module named 'binpacking'

In [64]:
# Cut List

stock_len = 10 * unit.feet
stock_len = stock_len.to('meters')

cut_list = []

def use_scrap(p_len):
    for j, pipe in enumerate(cut_list):
        scrap = pipe[1]
        if scrap > p_len:
            print(f'use scrap {cut_list[j]}')
            cut_list[j][1] = scrap - p_len
            cut_list[j][0].append(p_len)
            print(f'{cut_list[j]=}')
            return True
    return False
            

for p_len in comp_len.values():
    print(f'fit {p_len}')
    if use_scrap(p_len):
        next
    else:
        print(f'cut new pipe: {p_len} -| {stock_len - p_len}')
        cut_list.append([[p_len, ], stock_len - p_len])
        
{i: cuts for i, cuts in enumerate(cut_list)}

fit 2.2438545454545453 meter
cut new pipe: 2.2438545454545453 meter -| 0.8041454545454543 meter
fit 1.8705755083320594 meter
cut new pipe: 1.8705755083320594 meter -| 1.1774244916679402 meter
fit 1.6554144374829747 meter
cut new pipe: 1.6554144374829747 meter -| 1.392585562517025 meter
fit 1.5569240421702237 meter
cut new pipe: 1.5569240421702237 meter -| 1.4910759578297759 meter
fit 1.4637440116490719 meter
use scrap [[<Quantity(1.55692404, 'meter')>], <Quantity(1.49107596, 'meter')>]
cut_list[j]=[[<Quantity(1.55692404, 'meter')>, <Quantity(1.46374401, 'meter')>], <Quantity(0.0273319462, 'meter')>]
fit 1.214726530612245 meter
use scrap [[<Quantity(1.65541444, 'meter')>], <Quantity(1.39258556, 'meter')>]
cut_list[j]=[[<Quantity(1.65541444, 'meter')>, <Quantity(1.21472653, 'meter')>], <Quantity(0.177859032, 'meter')>]
fit 1.0711272727272727 meter
use scrap [[<Quantity(1.87057551, 'meter')>], <Quantity(1.17742449, 'meter')>]
cut_list[j]=[[<Quantity(1.87057551, 'meter')>, <Quantity(1.0711

{0: [[2.2438545454545453 <Unit('meter')>, 0.7769670503303139 <Unit('meter')>],
  0.027178404215140395 <Unit('meter')>],
 1: [[1.8705755083320594 <Unit('meter')>, 1.0711272727272727 <Unit('meter')>],
  0.10629721894066746 <Unit('meter')>],
 2: [[1.6554144374829747 <Unit('meter')>, 1.214726530612245 <Unit('meter')>],
  0.17785903190477992 <Unit('meter')>],
 3: [[1.5569240421702237 <Unit('meter')>, 1.4637440116490719 <Unit('meter')>],
  0.027331946180704003 <Unit('meter')>],
 4: [[0.8845631373748184 <Unit('meter')>,
   0.7276620210851118 <Unit('meter')>,
   0.6811194951762636 <Unit('meter')>,
   0.5565632653061224 <Unit('meter')>],
  0.19809208105768306 <Unit('meter')>]}

In [65]:
{i: [cut.to('feet') for cut in cuts[0]] for i, cuts in enumerate(cut_list)}

{0: [7.36172751133381 <Unit('foot')>, 2.549104495834363 <Unit('foot')>],
 1: [6.1370587543702735 <Unit('foot')>, 3.5141970890002385 <Unit('foot')>],
 2: [5.431149729274852 <Unit('foot')>, 3.985323263163534 <Unit('foot')>],
 3: [5.108018511057165 <Unit('foot')>, 4.802309749504829 <Unit('foot')>],
 4: [2.9021100307572785 <Unit('foot')>,
  2.3873425888619155 <Unit('foot')>,
  2.234644013045484 <Unit('foot')>,
  1.8259949649151 <Unit('foot')>]}